In [ ]:
!python -m pip install ultralytics==8.0.186 wandb==0.16.1

In [ ]:
#%env WANDB_SILENT=true
!wandb online

In [3]:
import wandb
import os

config = {
    "num_of_classes" : 1
}

#run = wandb.init(id="6cthhvso", resume="allow", project="YOLOv8")
run = wandb.init(project = "YOLOv8", job_type="train", config = config)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
from artifacts import download_artifact

artifact, dataset_location = download_artifact("vacocam_dataset", "latest", run=run)

wandb: Downloading large artifact vacocam_dataset:latest, 903.00MB. 3299 files... 
wandb:   3299 of 3299 files downloaded.  
Done. 0:0:32.6


In [5]:
from ultralytics import YOLO
from wandb.errors import CommError
import copy

model = YOLO("yolov8m.pt")

model.overrides['verbose'] = False

class YoloWandbCallback():
    def on_train_end(self, trainer):
        print("Training ended. Calling #on_train_end")
        self.save_vaco_model(trainer)
        
    def save_vaco_model(self, trainer):
        try:
            saved_artifact = run.use_artifact("vacocam_model:latest")

            draft_artifact = saved_artifact.new_draft()
            draft_artifact.medatata = vars(trainer.args)
            
            try:
                draft_artifact.remove("best.pt")
            except FileNotFoundError:
                pass
            
            try:
                draft_artifact.remove("last.pt")
            except FileNotFoundError:
                pass

        except CommError:
            draft_artifact = wandb.Artifact(
                f"vacocam_model", "model", metadata=vars(trainer.args)
            )

        if trainer.best.exists():
            draft_artifact.add_file(trainer.best)
        if trainer.last.exists():
            draft_artifact.add_file(trainer.last)

        run.log_artifact(draft_artifact, aliases=[f"run_{run.id}"])

wandb_callback = YoloWandbCallback()

model.add_callback("on_train_end", wandb_callback.on_train_end)

results = model.train(data = dataset_location + "/data.yaml",epochs = 600, imgsz=640, batch=36, patience=0)

New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186 🚀 Python-3.9.16 torch-1.12.1+cu116 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/notebooks/artifacts/vacocam_dataset:v15/data.yaml, epochs=600, patience=0, batch=36, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_c

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train11
Starting training for 600 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/600      14.8G      1.432      27.07      0.999         14        640: 100%|██████████| 47/47 [00:26<00:00,  1.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.63it/s]
                   all        238        180       0.56      0.374      0.424       0.25

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/600      14.4G      1.535      1.711      1.021          7        640: 100%|██████████| 47/47 [00:26<00:00,  1.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  2.28it/s]
                   all        238        180      0.392      0.211      0.221      0.109

    

Training ended. Calling #on_train_end


In [ ]:
from ultralytics import YOLO
from wandb.errors import CommError
import copy

model = YOLO("/notebooks/runs/detect/train10/weights/last.pt")

model.overrides['verbose'] = False

class YoloWandbCallback():
    def on_train_end(self, trainer):
        print("Training ended. Calling #on_train_end")
        self.save_vaco_model(trainer)
        
    def save_vaco_model(self, trainer):
        try:
            saved_artifact = run.use_artifact("vacocam_model:latest")

            draft_artifact = saved_artifact.new_draft()
            draft_artifact.medatata = vars(trainer.args)
            
            try:
                draft_artifact.remove("best.pt")
            except FileNotFoundError:
                pass
            
            try:
                draft_artifact.remove("last.pt")
            except FileNotFoundError:
                pass

        except CommError:
            draft_artifact = wandb.Artifact(
                f"vacocam_model", "model", metadata=vars(trainer.args)
            )

        if trainer.best.exists():
            draft_artifact.add_file(trainer.best)
        if trainer.last.exists():
            draft_artifact.add_file(trainer.last)

        run.log_artifact(draft_artifact, aliases=[f"run_{run.id}"])

wandb_callback = YoloWandbCallback()

model.add_callback("on_train_end", wandb_callback.on_train_end)

model.train(resume=True)

New https://pypi.org/project/ultralytics/8.0.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.186 🚀 Python-3.9.16 torch-1.12.1+cu116 CUDA:0 (NVIDIA RTX A4000, 16109MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=/notebooks/runs/detect/train10/weights/last.pt, data=/notebooks/artifacts/vacocam_dataset:v14/data.yaml, epochs=900, patience=0, batch=36, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_

  0%|          | 0.00/755k [00:00<?, ?B/s]

TensorBoard: Start with 'tensorboard --logdir runs/detect/train10', view at http://localhost:6006/

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]

Resuming training from /notebooks/runs/detect/train10/weights/last.pt from epoch 891 to 900 total epochs
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train10
Starting training for 900 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    891/900      13.9G      0.312     0.1775     0.7724         12        640: 100%|██████████| 39/39 [00:22<00:00,  1.71it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.74it/s]
                   all        217        180      0.919      0.692      0.813      0.603

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
    892/900      14.3G     0.3317     0.1847     0.7949         12        640: 100%|██████████| 39/39 [00:21<00:00,  1.84it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|████

In [ ]:
def save_artifact(run):
    saved_artifact = run.use_artifact("vacocam_model:latest")

    draft_artifact = saved_artifact.new_draft()

    try:
        draft_artifact.remove("best.pt")
    except FileNotFoundError:
        pass

    try:
        draft_artifact.remove("last.pt")
    except FileNotFoundError:
        pass

    draft_artifact.add_file("/notebooks/runs/detect/train8/weights/best.pt")
    draft_artifact.add_file("/notebooks/runs/detect/train8/weights/last.pt")

    run.log_artifact(draft_artifact, aliases=[f"run_{run.id}"])

#save_artifact(run)

In [ ]:
run.finish()

lr/pg0,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▁▄▄▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇█████▇██████████████
metrics/mAP50-95(B),▁▁▃▃▅▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████████
metrics/precision(B),▃▁▆▇▄▅▅▆▆▇▇▇▆▇▆▇▇▇▇█▇▇▇██▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇
metrics/recall(B),▁▁▃▃▆▇▆▆▇█▆▆▇▇▇▇▇▇▇▇███▇▇▇████▇████▇████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▆▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
